In [3]:
import numpy as np
import pandas as pd

In [10]:
# GDP 
# Employment to pop ratio
# Gov expenditure on education
# percentage of immigrants
# net migration

In [231]:
def CleanEconomic(data):
    
    # Read data
    economic_data = pd.read_csv(f'/Users/laurengilson/Desktop/perm_data/{data}.csv')
    
    # remove last 5 rows
    economic_data = economic_data[:-5]

    # drop unnecessary columns 
    economic_data.drop(['Series Name', 'Series Code', 'Country Name'], axis=1, inplace=True)
    
    # Turn to strings and replace '..' with '0'
    for column in list(economic_data.columns):
        economic_data[column] = list(map(str, economic_data[column]))
        economic_data[column] = [x.replace('..', '0') for x in economic_data[column]]
        
    if data == 'percentage_of_immigrants':
        
        economic_data['2007 [YR2007]'] = economic_data['2005 [YR2005]']
        economic_data.drop(['2005 [YR2005]', '2006 [YR2006]'], axis=1, inplace=True)
    
    elif data == 'net_migration':
        
        for idx, column in enumerate(list(economic_data.columns)[2:]):
            economic_data[column] = economic_data[column].astype(float)
            economic_data[column] = economic_data[column].replace(to_replace=0, value=economic_data.iloc[-1][column])
            economic_data[column] = economic_data[column].replace(0, np.nan) # replace 0 with NaNs
            economic_data.loc[economic_data[column].isnull(), column] = economic_data.iloc[:,economic_data.columns.get_loc(column)-1] # rest of nans with value from prev column
            economic_data[column] = economic_data[column].astype(float)
            
        economic_data.drop(['2002 [YR2002]', '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]'], axis=1, inplace=True)
        
    else: 
        None
    
    # dict of new columns to replace current column headers 
    years = ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
    col_list = list(economic_data.columns[1:])
    new_cols = dict(zip(col_list, years))
    
    # Change column head to fiscal years 
    economic_data.rename(columns=new_cols, inplace=True)
    
    
    if data == 'GDP_data':
        
        # Change 2019 - to have 3.1% growth on 2018.
        economic_data['2019'] = economic_data['2018'].astype(float) * 1.031
        
        # Put data in billions & round to 2dp 
        for column in list(economic_data.columns)[1:]:
            economic_data[column] = economic_data[column].astype(float)/1000000000 # Put in terms of billions
            economic_data[column] = economic_data[column].apply(lambda x: round(x, 2))
        
        # Transform df
        economic_data = pd.melt(economic_data, id_vars=['Country Code'], var_name='year', value_name='gdp')
        
        return economic_data
    
    elif data == 'employment_to_pop_ratio':
        
        for column in list(economic_data.columns)[1:]:
            economic_data[column] = economic_data[column].astype(float)
            economic_data[column] = economic_data[column].replace(to_replace=0, value=economic_data[column].mean())
            economic_data[column] = economic_data[column].apply(lambda x: round(x, 2))
           
        # Transform df
        economic_data = pd.melt(economic_data, id_vars=['Country Code'], var_name='year', value_name='employment')
        
        return economic_data
    
    elif data == 'gov_expenditure_education' or data == 'percentage_of_immigrants':

        for idx, column in enumerate(list(economic_data.columns)[1:]):
            economic_data[column] = economic_data[column].astype(float)
            economic_data[column] = economic_data[column].replace(to_replace=0, value=economic_data.iloc[-1][column]) # Replace zero with world average for that year
            economic_data[column] = economic_data[column].replace(0, np.nan) # replace 0 with NaNs
            economic_data.loc[economic_data[column].isnull(), column] = economic_data.iloc[:,economic_data.columns.get_loc(column)-1] # rest of nans with value from prev column
            economic_data[column] = economic_data[column].apply(lambda x: round(x, 2))
        
        # Transform df
        if data == 'gov_expenditure_education':
            economic_data = pd.melt(economic_data, id_vars=['Country Code'], var_name='year', value_name='gov_expen')
            return economic_data
        else:
            economic_data = pd.melt(economic_data, id_vars=['Country Code'], var_name='year', value_name='percentage_of_immigrants')
            return economic_data
    
    else: 
         # Transform df
        economic_data = pd.melt(economic_data, id_vars=['Country Code'], var_name='year', value_name='net_migration')
        return economic_data

In [232]:
economic_list = ['GDP_data','employment_to_pop_ratio','gov_expenditure_education','percentage_of_immigrants','net_migration']

In [233]:
CleanEconomic('net_migration')

,Country Code,year,net_migration
0,USA,2008,5033689.0
1,AFG,2008,-777497.0
2,ALB,2008,-217358.0
3,DZA,2008,-357340.0
4,ASM,2008,0.0
5,AND,2008,0.0
6,AGO,2008,85286.0
7,ATG,2008,-92.0
8,ARG,2008,-84998.0
9,ARM,2008,-183361.0


In [222]:
cleaned_dfs = []

for data in economic_list:
    cleaned_dfs.append(CleanEconomic(data))

In [228]:
for i, df in enumerate(cleaned_dfs):
    print(cleaned_dfs[i].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 3 columns):
Country Code    3168 non-null object
year            3168 non-null object
gdp             3168 non-null float64
dtypes: float64(1), object(2)
memory usage: 74.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 3 columns):
Country Code    3168 non-null object
year            3168 non-null object
employment      3168 non-null float64
dtypes: float64(1), object(2)
memory usage: 74.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 3 columns):
Country Code    3168 non-null object
year            3168 non-null object
gov_expen       3168 non-null float64
dtypes: float64(1), object(2)
memory usage: 74.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 3 columns):
Country Code                3168 non-null object
year             

In [218]:
exmpl = pd.read_csv(f'/Users/laurengilson/Desktop/perm_data/net_migration.csv')

In [219]:
exmpl.tail(10)

,Series Name,Series Code,Country Name,Country Code,2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
259,Net migration,SM.POP.NETM,Sub-Saharan Africa,SSF,-1080073,..,..,..,..,-1983360,...,..,..,..,-2046131,..,..,..,..,-1724595,..
260,Net migration,SM.POP.NETM,Sub-Saharan Africa (excluding high income),SSA,-1082586,..,..,..,..,-1980849,...,..,..,..,-2043930,..,..,..,..,-1723595,..
261,Net migration,SM.POP.NETM,Sub-Saharan Africa (IDA & IBRD countries),TSS,-1080073,..,..,..,..,-1983360,...,..,..,..,-2046131,..,..,..,..,-1724595,..
262,Net migration,SM.POP.NETM,Upper middle income,UMC,-4691170,..,..,..,..,-2520309,...,..,..,..,2780947,..,..,..,..,-525277,..
263,Net migration,SM.POP.NETM,World,WLD,0,..,..,..,..,0,...,..,..,..,0,..,..,..,..,0,..
264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,Last Updated: 01/30/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
